In [1]:
import numpy as np
# Set parameters
sigma = 0.2
r_ = 0
drift = r_ - 0.5 * sigma**2
T = 1
m = 10
delta_t = T / m
N = 10000000 # number of simulations
# Number of RV we simulate is N * m

In [2]:
# Generate paths, note we skip time 0
sims = np.random.normal(drift * delta_t, sigma * delta_t**0.5, (N,m)) # note scale param is sd not var
paths = np.cumsum(sims, axis=1)

res = np.any(paths > 0, axis=1)
prob = np.sum(~res) / N
sd = (prob * (1 - prob) / N)**0.5
print(f"Probability of maximum 0 is about {prob}, with sd {sd}")
print(f"Hence 95% CI: [{prob - 1.96 * sd}, {prob + 1.96 * sd}")

Probability of maximum 0 is about 0.1957671, with sd 0.00012547603060249796
Hence 95% CI: [0.1955211669800191, 0.1960130330199809


In [127]:
# Now we will estimate B2 of the paper Broadie 1999
# TODO research errors, sigma and T are fine, very high drift is also off
sigma = 0.5
T = 1
m = 200
M = 100000 # this is BM path, above is discrete
N = 1000

drift = 0.1 # Happens when r = 0.5 sigma^2
delta_t = T / M
draws = np.random.normal(drift * delta_t, sigma * delta_t**0.5, (N, M))
#draws = np.random.normal(0, 1, (N,M))
draws = np.c_[np.zeros(N), draws]
BM_path = np.cumsum(draws, axis=1)
max_bm = np.max(BM_path, axis=1)
watch_moments = range(0, M, M//m)
max_bw = np.max(BM_path[:, watch_moments], axis=1)
z = BM_path[:, watch_moments]
print(np.sum((max_bw == max_bm))) # too much I think
# print(np.sum(((max_bw == max_bm) & (max_bm != 0)))) # too much I think
print("Beta 1 should be 0.5826 and Beta 2 0.425")
B_1_est = m**0.5 * np.mean(max_bm-max_bw)/(sigma * T**0.5)
B_2_est = m*np.mean((max_bm - max_bw)**2)/(sigma**2 * T)
print(f"Beta 1: {B_1_est}, {np.std(max_bm- max_bw)/(N**0.5)}")
print(f"Beta 2: {B_2_est}")

3
Beta 1 should be 0.5826 and Beta 2 0.425
Beta 1: 0.5695982959811765, 0.00036813733606542943
Beta 2: 0.4328622973489406


In [129]:
# Here we will use control variates and estimate the constant c
from scipy.special import zeta
B_1 = -zeta(0.5)/np.sqrt(2 * np.pi)

new_b2_est = B_2_est + 0.5 * (B_1_est - B_1)